In [1]:
import pandas as pd
import glob
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# helper functions for log parsing

def pass_no_pass(lines):
    for line in lines:
        if "PASSED" in line:
            return True
        elif "FAILED" in line:
            return False
    return

def parse_fp_for_int_diff(lines):
    vals = {'expected': [], 'actual': []}
    delimiters = [" ", "\n"]
    for i in range(len(lines)):
        if "INT" in lines[i]:
            for delimiter in delimiters:
                line = " ".join(lines[i+1].split(delimiter))
            split = list(map(int, line.split()))
            if ("Expected value of" in lines[i]) and ("in" not in lines[i]):
                # print(lines[i])
                vals['expected'].append(split)
            elif "Last value of" in lines[i]:
                # print(lines[i])
                vals['actual'].append(split)

    diffs = []
    for i in range(len(vals['actual'])):
        for j in range(len(vals['actual'][i])):
            if vals['expected'][i][j] != vals['actual'][i][j]:
                diffs.append(int(vals['expected'][i][j]) - int(vals['actual'][i][j]))
    return diffs

In [3]:
# create csv of testing logs
# change bool to generate csv if not created
gen_csv = False

def generate_csv():

    df = pd.DataFrame(columns=["kind", "run", "log", "lines", "time", "name", "path", "iters", "expected_vs_actual", "diff_count", "passed"])

    for filename in glob.iglob('**', recursive=True):
        # skip exceptions, keep .txts with logs
        if "log" not in filename or filename[-4:] != ".txt" or "exception" in filename:
            continue
        tmp = filename.split('/')
        if not len(tmp) - 1:
            continue
        fp = open(filename, 'r')
        lines = fp.readlines()
        fp.close()

        kind, name = tmp[0], tmp[-1]

        # TODO: something with GXL logs
        if kind == "gxld":
            continue

        # get iteration count
        iter = [line for line in lines if "Exited" in line][0].split()[-1]
        # get count of i/o matches
        int_diff = parse_fp_for_int_diff(lines)
        # get pass/fail
        passed = pass_no_pass(lines)


        # parse datetime from file name
        try:
            time = datetime.strptime(filename[-21:-4], '%Y%m%d_%H-%M-%S') # everything but GXLD logs
        except Exception:
            try:
                time = datetime.strptime(filename[-18:-4], '%m-%d-%y_%H-%M') # specifically for GXLD logs
            except Exception as e:
                raise(e)
        
        # get 'run' and 'log' numbers
        runlog = name.split('_')[2]
        run, log = [int(x) for x in runlog.split('run')[-1].split('log')]

        row = {"kind": kind, "run":run, "log": log, "lines": len(lines), "iters": int(iter), "time": time, "name": name,\
            "path": filename, "expected_vs_actual": int_diff, "diff_count": len(int_diff), "passed": passed}
        if df.name.str.contains(name).any():
            continue
        else:
            df.loc[len(df), :] = row
    df = df.sort_values(by=['kind', 'run', 'log']).reset_index(drop=True)
    df.to_csv("./results.csv")
if gen_csv:
    generate_csv()
df = pd.read_csv("./results.csv")

In [4]:
nrows, ncols = 3072, 4096
true_bits = nrows * ncols
nbanks, nsections, row_per_sec = 8, 16, 14
_32k, vrs, vec_size = 32768, 14, 16
our_bits = _32k * vrs * vec_size
our_bits

7340032

In [11]:
df[(df.kind == "mmb_dynamic") & (df.diff_count > 0)].reset_index(drop=True)

,Unnamed: 0,kind,run,log,lines,time,name,path,iters,expected_vs_actual,diff_count,passed
0,34,mmb_dynamic,2,2,23,2023-11-13 03:17:24,log_dynamicMMBtest_run2log2_20231113_03-17-24.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run2log2_...,1,"[26925, 24797, 47441, 27959, 39926, 45158, 248...",32768,True
1,35,mmb_dynamic,2,2,23,2023-11-13 03:52:52,log_dynamicMMBtest_run2log2_20231113_03-52-52.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run2log2_...,1,"[26925, 24797, 47441, 27959, 39926, 45158, 248...",32768,True
2,36,mmb_dynamic,2,2,23,2023-11-13 03:24:03,log_dynamicMMBtest_run2log2_20231113_03-24-03.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run2log2_...,1,"[26925, 24797, 47441, 27959, 39926, 45158, 248...",32768,True
3,51,mmb_dynamic,2,7,73,2023-11-13 03:24:09,log_dynamicMMBtest_run2log7_20231113_03-24-09.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run2log7_...,6,[2048],1,False
4,52,mmb_dynamic,2,8,73,2023-11-13 03:24:10,log_dynamicMMBtest_run2log8_20231113_03-24-10.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run2log8_...,7,"[64, 2048, 128, 2, 8192]",5,False
...,...,...,...,...,...,...,...,...,...,...,...,...
631,696,mmb_dynamic,4,5,73,2023-11-13 02:53:39,log_dynamicMMBtest_run4log5_20231113_02-53-39.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run4log5_...,56,"[-1536, 256, 2560, -4608, 3072, -3840, -3840, ...",32709,False
632,697,mmb_dynamic,6,6,72,2023-11-13 02:58:26,log_dynamicMMBtest_run6log6_20231113_02-58-26.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run6log6_...,10,"[512, 32768, 4096, 4096, 16, 32768, 1024, 256,...",68,False
633,698,mmb_dynamic,7,7,72,2023-11-13 02:59:23,log_dynamicMMBtest_run7log7_20231113_02-59-23.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run7log7_...,10,"[512, 32768, 4096, 4096, 16, 32768, 1024, 256,...",68,False
634,699,mmb_dynamic,8,8,72,2023-11-13 03:04:20,log_dynamicMMBtest_run8log8_20231113_03-04-20.txt,mmb_dynamic/_logs/log_dynamicMMBtest_run8log8_...,54,"[-14567, 16071, 5778, 7984, -12062, -20785, 11...",4126,False


In [7]:
masks = []
for path in df[df.kind == "mmb_dynamic"].path:
    lines = open(path).read().split('\n')
    mask, mask_bool = [], False
    for line in lines:
        if "mask" in line:
            mask_bool = True
            continue
        elif ("SECTION" in line) or (line == ""):
            mask_bool = False
        if mask_bool:
            mask.append(line)
    if len(mask): masks.append(mask)

In [114]:
f = open('/home/jacob/radTestLogBackup/mmb_static/log_staticMMBtest_run2log4_20231113_06-13-00.txt')
lines = f.read().split("\n")
f.close()

In [121]:
expected, actual = [], []
delimiters = [" ", "\n"]
for i in range(len(lines)):
    if "INT" in lines[i]:
        for delimiter in delimiters:
            line = " ".join(lines[i+1].split(delimiter))
        split = list(map(int, line.split()))
        split = list(map(lambda x: '{0:016b}'.format(x), split))
        if ("Expected value of" in lines[i]) and ("in" not in lines[i]):
            # print(lines[i])
            expected.append(split)
        elif "Last value of" in lines[i]:
            # print(lines[i])
            actual.append(split)
if not len(expected)-1:
    expected, actual = expected[0], actual[0]

In [19]:
! rm test_bits.txt
f = open("test_bits.txt", "a")
for i in range(len(masks[632])):
    f.write(masks[632][i])
    if i < len(masks[632]) - 1:
        f.write('\n')
f.close()

In [16]:
len(masks[632]), len(masks[632][0])

(48, 32768)

In [123]:
len(expected), len(expected[0])

(32768, 16)

In [205]:
_32k * 16

524288

In [207]:
ncols * nbanks * nsections

131072